In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import numpy as np
import pandas as pd
import csv
import time
import torch
import sklearn

from sklearn.metrics import classification_report
from torch import nn
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from skimage import io, transform
from torchvision import datasets, models, transforms

from utils import label_map_util
from utils import visualization_utils as vis_util

In [3]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = \
    'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

result_categories = ['4/5', '2/3', '>5']

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map,
        max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [5]:
def get_max_num(id):
    try:
        images = os.listdir("Myauto_data/Car_Images/Car_Images/{}".format(id))
    except:
        return 0
    return len(images)

In [6]:
def get_tensor_image(path):
    path = "/Users/andghuladze/Documents/Vision/Myauto_data/Car_Images/Car_Images/{}".format(path)
    image = transform.resize(io.imread(path), (224, 224))/255
    image = image[:, :, :3]

    sample = torch.Tensor(np.einsum('ijk->kij', image))

    return sample

In [7]:
def object_detection_function(id):
    total_passed_vehicle = 0
    speed = 'waiting...'
    direction = 'waiting...'
    size = 'waiting...'
    color = 'waiting...'
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            car_data = []
            img_number = get_max_num(id)
            for i in range(1, img_number + 1):
                input_frame = cv2.imread("Myauto_data/Car_Images/Car_Images/{}/{}.jpg".format(id, i))

                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(input_frame, axis=0)

                # Actual detection.
                (boxes, scores, classes, num) = \
                    sess.run([detection_boxes, detection_scores,
                             detection_classes, num_detections],
                             feed_dict={image_tensor: image_np_expanded})

                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    0,
                    input_frame,
                    boxes[0],
                    classes[0].astype(np.int32),
                    scores[0],
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=4
                    )

                for j in range(len(classes[0])):
                    if classes[0][j] == 3 or classes[0][j] == 8:
                        if scores[0][j] > 0.5:
                            car_data.append(get_tensor_image(str(id) + "/" + str(i) + ".jpg"))
                            break
                            
            return car_data

In [8]:
class model_init(nn.Module):
    def __init__(self):
        super(model_init, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(models.resnet18(pretrained=True))
        self.layers.append(nn.Linear(1000, 512)) 
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(512, 64))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(64, 3))
        self.layers.append(nn.Softmax())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [9]:
model = model_init()
model.load_state_dict(torch.load('models/doors_model.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
doors_csv = pd.read_csv("validation/doors_validation.csv")
original_result = []
model_result = []

for i in range(len(doors_csv)):
    car_data = get_tensor_image(doors_csv.iloc[i].img_path[1:])
    car_data = torch.stack([car_data])
    result = model(car_data)
    original_result.append(result_categories.index(doors_csv.iloc[i].Doors))
    model_result = result.argmax().numpy()
    
print(classification_report(original_result, model_result))
# car_data = torch.stack(object_detection_function(45801765))
# result = model(car_data)
# print(result)

/Users/andghuladze/Python/venv_3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


In [133]:
sum = result.sum(dim = 0)
best_assumption = result_categories[sum.argmax()]

In [134]:
print("CAR DOORS NUM: " + best_assumption)

CAR DOORS NUM: 4/5
